<a href="https://colab.research.google.com/github/afzal/MiniTalky/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline

# Load the model
model = pipeline("text-generation", model="distilgpt2")

# Chat loop
while True:
    prompt = input("You: ")
    result = model(prompt, max_length=50, temperature=0.7, top_k=50)
    print("Bot:", result[0]["generated_text"])

In [ ]:
from transformers import pipeline

# Load the model
model = pipeline("text-generation", model="distilgpt2")

# Chat loop
while True:
    prompt = input("You: ")
    result = model(prompt, max_length=50, temperature=0.7, top_k=40, truncation=True)
    # Remove the prompt from the reply
    reply = result[0]["generated_text"].replace(prompt, "").strip()
    print("Bot:", reply)

In [ ]:
from transformers import pipeline

# Load the model
model = pipeline("text-generation", model="gpt2")

# Chat loop
while True:
    prompt = input("You: ")
    if not prompt.strip():  # If input is empty, skip
        print("Bot: Say something!")
        continue
    full_prompt = f"Human: {prompt} Bot:"  # Add context
    result = model(full_prompt, max_length=50, temperature=0.9, top_k=50, truncation=True)
    reply = result[0]["generated_text"].replace(full_prompt, "").strip()
    print("Bot:", reply)